# Load Libraries

In [ ]:
from google.colab import drive
import os
import pandas as pd
from google.colab import files
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats.mstats import winsorize

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load and Merge Data

In [ ]:
returns = pd.read_csv('temp1.csv')
returns

,TICKER,RetEarly2020,RetLate2020
0,JJSF,-0.340234493,0.300340
1,PLXS,-0.290875889,0.433469
2,HNGR,-0.435711705,0.411425
3,ORCL,-0.08359557,0.356166
4,MSFT,0.003212116,0.421139
...,...,...,...
2711,SIX,-0.716470643,1.719300
2712,VPG,-0.409412013,0.567728
2713,FN,-0.158545179,0.422103
2714,CBOE,-0.253901378,0.056516


In [ ]:
df = pd.read_csv('temp_NAICS.csv')
len(df.NAICS.unique())

538

In [ ]:
df1 = pd.read_csv('tempSIC.csv')
len(df1.SIC.unique())

351

## NAICS Table 

In [ ]:
#For better analysis, I reduced the industry codes to only include 25 industries for both NAICS and SIC. 
df_25_cnt = df.groupby('NAICS')['NAICS'].count().sort_values(ascending=False)[:25]
df_25_cnt = pd.DataFrame(df_25_cnt)
df_25_cnt.rename(columns={'NAICS': 'Count'}, inplace=True)
df_25_cnt.reset_index(inplace=True)

naics_list = []
for i in df_25_cnt.NAICS:
  naics_list.append(i)

naics_25 = df[df['NAICS'].isin(naics_list)]

In [ ]:
naics_25_returns = pd.merge(returns, naics_25, on='TICKER')
naics_25_returns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1316 entries, 0 to 1315
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        1316 non-null   object 
 1   RetEarly2020  1316 non-null   object 
 2   RetLate2020   1316 non-null   float64
 3   NAICS         1316 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 51.4+ KB


Since the datatype of the RetEarly2020 column is object, we need to adjust it to float64 for better statistical summary analysis. 

In [ ]:
#get ride of null value 
naics_25_returns.drop(naics_25_returns.loc[naics_25_returns['RetEarly2020']  == '#VALUE!'].index, inplace=True)
#convert the RetEarly2020 column to numeric
naics_25_returns['RetEarly2020'] = pd.to_numeric(naics_25_returns['RetEarly2020'])

naics_25_returns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1315
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        1314 non-null   object 
 1   RetEarly2020  1314 non-null   float64
 2   RetLate2020   1314 non-null   float64
 3   NAICS         1314 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 51.3+ KB


In [ ]:
naics_25_returns

,TICKER,RetEarly2020,RetLate2020,NAICS
0,ORCL,-0.083596,0.356166,519130
1,MSFT,0.003212,0.421139,511210
2,RGEN,0.043676,0.984981,325414
3,PTVCB,-0.139439,0.018376,524126
4,INDB,-0.221261,0.159762,522110
...,...,...,...,...
1311,CDXS,-0.302063,0.956094,325414
1312,SPSC,-0.160772,1.334771,519130
1313,CRMD,-0.506867,1.069637,325414
1314,HPP,-0.319787,-0.022453,531120


## SIC Table

In [ ]:
df1_25_cnt = df1.groupby('SIC')['SIC'].count().sort_values(ascending=False)[:25]
df1_25_cnt = pd.DataFrame(df1_25_cnt)
df1_25_cnt.rename(columns={'SIC': 'Count'}, inplace=True)
df1_25_cnt.reset_index(inplace=True)

sic_list = []
for i in df1_25_cnt.SIC:
  sic_list.append(i)

sic_25 = df1[df1['SIC'].isin(sic_list)]

In [ ]:
sic_25_returns = pd.merge(returns, sic_25, on='TICKER')
sic_25_returns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1395 entries, 0 to 1394
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        1395 non-null   object 
 1   RetEarly2020  1395 non-null   object 
 2   RetLate2020   1395 non-null   float64
 3   SIC           1395 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 54.5+ KB


Since the datatype of the RetEarly2020 column is object, we need to adjust it to float64 for better statistical summary analysis.

In [ ]:
#get ride of null value 
sic_25_returns.drop(sic_25_returns.loc[sic_25_returns['RetEarly2020']  == '#VALUE!'].index, inplace=True)
#convert the RetEarly2020 column to numeric
sic_25_returns['RetEarly2020'] = pd.to_numeric(sic_25_returns['RetEarly2020'])

sic_25_returns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393 entries, 0 to 1394
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        1393 non-null   object 
 1   RetEarly2020  1393 non-null   float64
 2   RetLate2020   1393 non-null   float64
 3   SIC           1393 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 54.4+ KB


In [ ]:
sic_25_returns

,TICKER,RetEarly2020,RetLate2020,SIC
0,ORCL,-0.083596,0.356166,7370
1,MSFT,0.003212,0.421139,7372
2,TROW,-0.191152,0.581942,6282
3,RGEN,0.043676,0.984981,2836
4,PTVCB,-0.139439,0.018376,6331
...,...,...,...,...
1390,CDXS,-0.302063,0.956094,2836
1391,SPSC,-0.160772,1.334771,7370
1392,CRMD,-0.506867,1.069637,2836
1393,HPP,-0.319787,-0.022453,6798


## GGROUP Table

In [ ]:
ggroup_returns = pd.read_csv('assign4.csv')
ggroup_returns.drop(columns=['Unnamed: 0'], inplace=True)
ggroup_returns

,TICKER,RetEarly2020,RetLate2020,GGROUP
0,JJSF,-0.340234,0.300340,3020
1,PLXS,-0.290876,0.433469,4520
2,HNGR,-0.435712,0.411425,3510
3,ORCL,-0.083596,0.356166,4510
4,MSFT,0.003212,0.421139,4510
...,...,...,...,...
2661,SIX,-0.716471,1.719300,2530
2662,VPG,-0.409412,0.567728,4520
2663,FN,-0.158545,0.422103,4520
2664,CBOE,-0.253901,0.056516,4020


## GSECTOR Table

In [ ]:
gs = pd.read_csv('tempGICS_sector.csv')
gsector_returns = pd.merge(returns, gs, on='TICKER')
gsector_returns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2671 entries, 0 to 2670
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        2671 non-null   object 
 1   RetEarly2020  2671 non-null   object 
 2   RetLate2020   2671 non-null   float64
 3   GSECTOR       2671 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 104.3+ KB


Same as before, as the datatype of the RetEarly2020 column is object, we need to adjust it to float64 for better statistical summary analysis.

In [ ]:
#get ride of null value 
gsector_returns.drop(gsector_returns.loc[gsector_returns['RetEarly2020']  == '#VALUE!'].index, inplace=True)
#convert the RetEarly2020 column to numeric
gsector_returns['RetEarly2020'] = pd.to_numeric(gsector_returns['RetEarly2020'])

gsector_returns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2670
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        2666 non-null   object 
 1   RetEarly2020  2666 non-null   float64
 2   RetLate2020   2666 non-null   float64
 3   GSECTOR       2666 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 104.1+ KB


In [ ]:
gsector_returns

,TICKER,RetEarly2020,RetLate2020,GSECTOR
0,JJSF,-0.340234,0.300340,30
1,PLXS,-0.290876,0.433469,45
2,HNGR,-0.435712,0.411425,35
3,ORCL,-0.083596,0.356166,45
4,MSFT,0.003212,0.421139,45
...,...,...,...,...
2666,SIX,-0.716471,1.719300,25
2667,VPG,-0.409412,0.567728,45
2668,FN,-0.158545,0.422103,45
2669,CBOE,-0.253901,0.056516,40


For this project, I am going to looking at three main aspects that could explain varying exposures to Covid-19 Market Shock for both early and late 2020. They are **industries&sectors**, **textual analysis of company business models** and **financial accounting characteristics**.  

# Does Industry or Sector Explain Differences in Stock Returns for Early and Late 2020?

For better analysis, for NAICS and SIC we only chose the top 25 industries that have the most companies.

## Average Stock Returns for Each Industry

### NAICS

In [ ]:
#ReturnEarly2020
for i in naics_25_returns.NAICS.unique():
  print(f"The average stock return for industry {i} is: {round(naics_25_returns[naics_25_returns['NAICS'] == i].RetEarly2020.mean(), 3)}.")

The average stock return for industry 519130 is: -0.133.
The average stock return for industry 511210 is: -0.145.
The average stock return for industry 325414 is: -0.106.
The average stock return for industry 524126 is: -0.245.
The average stock return for industry 522110 is: -0.372.
The average stock return for industry 325412 is: -0.232.
The average stock return for industry 531120 is: -0.336.
The average stock return for industry 325413 is: 0.187.
The average stock return for industry 518210 is: -0.235.
The average stock return for industry 541512 is: -0.173.
The average stock return for industry 522120 is: -0.324.
The average stock return for industry 339113 is: -0.294.
The average stock return for industry 334413 is: -0.229.
The average stock return for industry 523920 is: -0.342.
The average stock return for industry 22111 is: -0.179.
The average stock return for industry 334510 is: -0.154.
The average stock return for industry 339112 is: -0.099.
The average stock return for indu

In [ ]:
list1 = []
for i in naics_25_returns.NAICS.unique():
    list1.append((i,round(naics_25_returns[naics_25_returns['NAICS'] == i].RetEarly2020.mean(), 3)))

In [ ]:
early_2020 = pd.DataFrame(list1, columns=['NAICS', 'AVG'])
early_2020[early_2020['AVG'] == max(early_2020['AVG'])]

,NAICS,AVG
7,325413,0.187


The highest return industry for early 2020 is In-Vitro Diagnostic Substance Manufacturing. The rest of the industries have the negative average returns for early 2020, only the In-Vitro Diagnostic Substance Manufacturing has the positive early 2020 returns. This makes sense because the economy got hit hard during covid and every industry is on the decline, but this particular industry is resilient and specially needed to manufacture covid need medical solutions. 



In [ ]:
early_2020[early_2020['AVG'] == min(early_2020['AVG'])]

,NAICS,AVG
22,2111,-0.63


The lowest return industry for early 2020 is Oil and Gas Extraction. Because of the quarantine, people rarely went out for trips or work. They had to stay at home to avoid getting the covid. People barely used the car and consumed gas, the demand went down and that's why the energy industry was going down at that time and has the lowest return for early 2020 among all other industries.


In [ ]:
#ReturnLate2020
for i in naics_25_returns.NAICS.unique():
  print(f"The average stock return for industry {i} is: {round(naics_25_returns[naics_25_returns['NAICS'] == i].RetLate2020.mean(), 3)}.")

The average stock return for industry 519130 is: 1.093.
The average stock return for industry 511210 is: 1.077.
The average stock return for industry 325414 is: 0.793.
The average stock return for industry 524126 is: 0.219.
The average stock return for industry 522110 is: 0.424.
The average stock return for industry 325412 is: 0.351.
The average stock return for industry 531120 is: 0.371.
The average stock return for industry 325413 is: 1.013.
The average stock return for industry 518210 is: 1.166.
The average stock return for industry 541512 is: 0.559.
The average stock return for industry 522120 is: 0.323.
The average stock return for industry 339113 is: 0.737.
The average stock return for industry 334413 is: 1.433.
The average stock return for industry 523920 is: 0.837.
The average stock return for industry 22111 is: 0.133.
The average stock return for industry 334510 is: 0.732.
The average stock return for industry 339112 is: 0.414.
The average stock return for industry 2211 is: 0.

In [ ]:
list2 = []
for i in naics_25_returns.NAICS.unique():
    list2.append((i,round(naics_25_returns[naics_25_returns['NAICS'] == i].RetLate2020.mean(), 3)))

In [ ]:
late_2020 = pd.DataFrame(list2, columns=['NAICS', 'AVG'])
late_2020[late_2020['AVG'] == max(late_2020['AVG'])]

,NAICS,AVG
22,2111,1.729


The Oil and Gas Extraction industry became the highest return industry for late 2020. Since almost all of the industries have the positive average return for late 2020, it seems like the economy was gradually recovering from the covid hit. People started to going out and traveling. Since the covid still didn't die down and traveling by plane or any other public transportations was restrcited and have the potential risk to be exposed to covid, people chose to travel by car and all of gas and oil demand has gone up. That's why the Oil and Gas Extraction industry were doing great for the late 2020.

In [ ]:
late_2020[late_2020['AVG'] == min(late_2020['AVG'])]

,NAICS,AVG
19,483111,-0.019


The Deep Sea Freight Transportation industry got hit the worst by the covid in late 2020. It has the lowest return among other industries. While other industries have the positive returns for late 2020, it has the negative return. The global trade demand got hit really bad by the global pandenmic. The trade import from big manufacturer countries like India and China decreased a lot compared to the previous year. 


### SIC

In [ ]:
#ReturnEarly2020
for i in sic_25_returns.SIC.unique():
  print(f"The average stock return for industry {i} is: {round(sic_25_returns[sic_25_returns['SIC'] == i].RetEarly2020.mean(), 3)}.")

The average stock return for industry 7370 is: -0.166.
The average stock return for industry 7372 is: -0.142.
The average stock return for industry 6282 is: -0.294.
The average stock return for industry 2836 is: -0.106.
The average stock return for industry 6331 is: -0.219.
The average stock return for industry 6020 is: -0.371.
The average stock return for industry 2834 is: -0.232.
The average stock return for industry 2835 is: 0.187.
The average stock return for industry 7374 is: -0.289.
The average stock return for industry 7373 is: -0.141.
The average stock return for industry 6035 is: -0.317.
The average stock return for industry 3842 is: -0.307.
The average stock return for industry 3674 is: -0.229.
The average stock return for industry 5500 is: -0.426.
The average stock return for industry 4931 is: -0.168.
The average stock return for industry 3845 is: -0.149.
The average stock return for industry 3841 is: -0.099.
The average stock return for industry 6798 is: -0.36.
The average 

In [ ]:
list3 = []
for i in sic_25_returns.SIC.unique():
    list3.append((i,round(sic_25_returns[sic_25_returns['SIC'] == i].RetEarly2020.mean(), 3)))

In [ ]:
early_2020_sic = pd.DataFrame(list3, columns=['SIC', 'AVG'])
early_2020_sic[early_2020_sic['AVG'] == max(early_2020_sic['AVG'])]

,SIC,AVG
7,2835,0.187


Similar with the NAICS industry code, the highest return industry of the SIC code for early 2020 is In Vitro and In Vivo Diagnostic Substances. The rest of the industries have the negative average returns for early 2020, only the In Vitro and In Vivo Diagnostic Substances industry has the positive early 2020 returns. This makes sense because the economy got hit hard during covid and every industry is on the decline, but this particular industry is resilient and specially needed to manufacture covid need medical solutions.


In [ ]:
early_2020_sic[early_2020_sic['AVG'] == min(early_2020_sic['AVG'])]

,SIC,AVG
19,1311,-0.673


Similar with the NAICS industry code, the lowest return industry of the SIC code for early 2020 is Crude Petroleum and Natural Gas. Because of the quarantine, people rarely went out for trips or work. They had to stay at home to avoid getting the covid. People barely used the car and consumed gas, the demand went down and that's why the energy industry was going down at that time and has the lowest return for early 2020 among all other industries.

In [ ]:
#ReturnLate2020
for i in sic_25_returns.SIC.unique():
  print(f"The average stock return for industry {i} is: {round(sic_25_returns[sic_25_returns['SIC'] == i].RetLate2020.mean(), 3)}.")

The average stock return for industry 7370 is: 1.15.
The average stock return for industry 7372 is: 1.082.
The average stock return for industry 6282 is: 0.773.
The average stock return for industry 2836 is: 0.793.
The average stock return for industry 6331 is: 0.2.
The average stock return for industry 6020 is: 0.422.
The average stock return for industry 2834 is: 0.351.
The average stock return for industry 2835 is: 1.013.
The average stock return for industry 7374 is: 0.765.
The average stock return for industry 7373 is: 0.504.
The average stock return for industry 6035 is: 0.294.
The average stock return for industry 3842 is: 0.767.
The average stock return for industry 3674 is: 1.433.
The average stock return for industry 5500 is: 1.732.
The average stock return for industry 4931 is: 0.145.
The average stock return for industry 3845 is: 0.737.
The average stock return for industry 3841 is: 0.414.
The average stock return for industry 6798 is: 0.444.
The average stock return for in

Compared to the NAICS industry code, average late 2020 returns of the SIC industry code do not have the negative value. All 25 industries were doing great for the late 2020.

In [ ]:
list4 = []
for i in sic_25_returns.SIC.unique():
    list4.append((i,round(sic_25_returns[sic_25_returns['SIC'] == i].RetLate2020.mean(), 3)))

In [ ]:
late_2020_sic = pd.DataFrame(list4, columns=['SIC', 'AVG'])
late_2020_sic[late_2020_sic['AVG'] == max(late_2020_sic['AVG'])]

,SIC,AVG
13,5500,1.732


Similar with the NAISC code, for SIC code the Retail-Auto Dealers & Gasoline Stations industry became the highest return industry for late 2020. Since almost all of the industries have the positive average return for late 2020, it seems like the economy was gradually recovering from the covid hit. People started to going out and traveling. Since the covid still didn't die down and traveling by plane or any other public transportations was restrcited and have the potential risk to be exposed to covid, people chose to travel by car and all of car and gas demand has gone up. That's why the Retail-Auto Dealers & Gasoline Stations industry were doing great for the late 2020.





In [ ]:
late_2020_sic[late_2020_sic['AVG'] == min(late_2020_sic['AVG'])]

,SIC,AVG
14,4931,0.145


The lowest return industry for late 2020 is Electric and Other Services Combined industry. This makes economic sense because electric services is sensitive to the health of the economy. Covid hits the economy very bad, lots of factries shut down or experienced downturn due to the prolonged lockdown. Malls were forced to close to avoid the spread of covid. In addition, airports and railways are major consumers of this industry, and the demand from them dropped sharply due to the lockdowns, which led to the overall revenue decrease and recover the slowest in the industry.

### GGROUP

In [ ]:
#ReturnEarly2020
for i in ggroup_returns.GGROUP.unique():
  print(f"The average stock return for industry {i} is: {round(ggroup_returns[ggroup_returns['GGROUP'] == i].RetEarly2020.mean(), 3)}.")

The average stock return for industry 3020 is: -0.181.
The average stock return for industry 4520 is: -0.29.
The average stock return for industry 3510 is: -0.134.
The average stock return for industry 4510 is: -0.177.
The average stock return for industry 4020 is: -0.36.
The average stock return for industry 2010 is: -0.317.
The average stock return for industry 3520 is: -0.141.
The average stock return for industry 4030 is: -0.249.
The average stock return for industry 4010 is: -0.368.
The average stock return for industry 2550 is: -0.385.
The average stock return for industry 1510 is: -0.351.
The average stock return for industry 5020 is: -0.341.
The average stock return for industry 2030 is: -0.327.
The average stock return for industry 6010 is: -0.328.
The average stock return for industry 2520 is: -0.392.
The average stock return for industry 4530 is: -0.227.
The average stock return for industry 5510 is: -0.148.
The average stock return for industry 2020 is: -0.333.
The average 

In [ ]:
list5 = []
for i in ggroup_returns.GGROUP.unique():
    list5.append((i,round(ggroup_returns[ggroup_returns['GGROUP'] == i].RetEarly2020.mean(), 3)))

In [ ]:
early_2020_ggroup = pd.DataFrame(list5, columns=['GGROUP', 'AVG'])
early_2020_ggroup[early_2020_ggroup['AVG'] == max(early_2020_ggroup['AVG'])]

,GGROUP,AVG
23,5010,-0.089


The highest return industry for early 2020 is GGROUP 5010 (Telecommunication Services). Even though every industry has the negative average returns for early 2020, the telecommunication services industry still doing better than other industries. This makes sense because the economy got hit hard during covid and every industry is on the decline, but people still need to use telecommunication services like phone services and data/internet services especially during quarantine period.

In [ ]:
early_2020_ggroup[early_2020_ggroup['AVG'] == min(early_2020_ggroup['AVG'])]

,GGROUP,AVG
19,1010,-0.581


The lowest return industry for early 2020 is GGROUP 1010 (Energy). Because of the quarantine, people rarely went out for trips or work. They had to stay at home to avoid getting the covid. People barely used the car and consumed gas, the demand went down and that's why the energy industry was going down at that time and has the lowest return for early 2020 among all other industries.

In [ ]:
#ReturnLate2020
for i in ggroup_returns.GGROUP.unique():
  print(f"The average stock return for industry {i} is: {round(ggroup_returns[ggroup_returns['GGROUP'] == i].RetLate2020.mean(), 3)}.")

The average stock return for industry 3020 is: 0.533.
The average stock return for industry 4520 is: 0.817.
The average stock return for industry 3510 is: 0.74.
The average stock return for industry 4510 is: 1.027.
The average stock return for industry 4020 is: 0.746.
The average stock return for industry 2010 is: 0.898.
The average stock return for industry 3520 is: 0.74.
The average stock return for industry 4030 is: 0.358.
The average stock return for industry 4010 is: 0.434.
The average stock return for industry 2550 is: 1.78.
The average stock return for industry 1510 is: 0.873.
The average stock return for industry 5020 is: 0.855.
The average stock return for industry 2030 is: 0.71.
The average stock return for industry 6010 is: 0.457.
The average stock return for industry 2520 is: 1.594.
The average stock return for industry 4530 is: 1.219.
The average stock return for industry 5510 is: 0.181.
The average stock return for industry 2020 is: 0.605.
The average stock return for ind

In [ ]:
list6 = []
for i in ggroup_returns.GGROUP.unique():
    list6.append((i,round(ggroup_returns[ggroup_returns['GGROUP'] == i].RetLate2020.mean(), 3)))

In [ ]:
late_2020_ggroup = pd.DataFrame(list6, columns=['GGROUP', 'AVG'])
late_2020_ggroup[late_2020_ggroup['AVG'] == max(late_2020_ggroup['AVG'])]

,GGROUP,AVG
22,2510,1.811


The highest return industry for late 2020 is GGROUP 2510 (Automobiles & Components). Since all the industries have the positive average return for late 2020, it seems like the economy was gradually recovering from the covid hit. People started to going out and traveling. Since the covid still didn't die down and traveling by plane or any other public transportations was restrcited and have the potential risk to be exposed to covid, people chose to travel by car and all of a sudden car demand has gone up. That's why the Automobiles & Components industry were doing great for the late 2020.

In [ ]:
late_2020_ggroup[late_2020_ggroup['AVG'] == min(late_2020_ggroup['AVG'])]

,GGROUP,AVG
16,5510,0.181


The lowest return industry for late 2020 is GGROUP 5510 (Utilities). This makes economic sense because utilities is sensitive to the health of the economy. Covid hits the economy very bad, lots of factries shut down or experienced downturn due to the prolonged lockdown. Malls were forced to close to avoid the spread of covid. In addition, airports and railways are major consumers of this industry, and the demand from them dropped sharply due to the lockdowns, which led to the over revenue decrease in the industry.

### GSECTOR

In [ ]:
#ReturnEarly2020
for i in gsector_returns.GSECTOR.unique():
  print(f"The average stock return for industry {i} is: {round(gsector_returns[gsector_returns['GSECTOR'] == i].RetEarly2020.mean(), 3)}.")

The average stock return for industry 30 is: -0.191.
The average stock return for industry 45 is: -0.217.
The average stock return for industry 35 is: -0.139.
The average stock return for industry 40 is: -0.346.
The average stock return for industry 20 is: -0.322.
The average stock return for industry 25 is: -0.41.
The average stock return for industry 15 is: -0.351.
The average stock return for industry 50 is: -0.288.
The average stock return for industry 60 is: -0.328.
The average stock return for industry 55 is: -0.148.
The average stock return for industry 10 is: -0.581.


In [ ]:
list7 = []
for i in gsector_returns.GSECTOR.unique():
    list7.append((i,round(gsector_returns[gsector_returns['GSECTOR'] == i].RetEarly2020.mean(), 3)))

In [ ]:
early_2020_gsector = pd.DataFrame(list7, columns=['GSECTOR', 'AVG'])
early_2020_gsector[early_2020_gsector['AVG'] == max(early_2020_gsector['AVG'])]

,GSECTOR,AVG
2,35,-0.139


The highest return industry for early 2020 is Heath Care industry. Even though every industry has the negative average returns for early 2020, the health care industry still doing better than other industries. This makes sense because the economy got hit hard during covid and every industry is on the decline, but a huge amount of people who got the covid virus need to be medically treated. That's why when other industries were doing so bad during early 2020 (when covid just hit), the health care industry were hit the least by the global pandemic. 

In [ ]:
early_2020_gsector[early_2020_gsector['AVG'] == min(early_2020_gsector['AVG'])]

,GSECTOR,AVG
10,10,-0.581


Same as the GGROUP industry code, for GSECTOR the lowest return industry for early 2020 is the Energy industry. Because of the quarantine, people rarely went out for trips or work. They had to stay at home to avoid getting the covid. People barely used the car and consumed gas, factories shut down and reduce the energy use, the demand went down and that's why the energy industry was going down at that time and has the lowest return for early 2020 among all other industries.



In [ ]:
#ReturnLate2020
for i in gsector_returns.GSECTOR.unique():
  print(f"The average stock return for industry {i} is: {round(gsector_returns[gsector_returns['GSECTOR'] == i].RetLate2020.mean(), 3)}.")

The average stock return for industry 30 is: 0.524.
The average stock return for industry 45 is: 1.005.
The average stock return for industry 35 is: 0.74.
The average stock return for industry 40 is: 0.498.
The average stock return for industry 20 is: 0.805.
The average stock return for industry 25 is: 1.541.
The average stock return for industry 15 is: 0.873.
The average stock return for industry 50 is: 0.778.
The average stock return for industry 60 is: 0.457.
The average stock return for industry 55 is: 0.181.
The average stock return for industry 10 is: 1.018.


In [ ]:
list8 = []
for i in gsector_returns.GSECTOR.unique():
    list8.append((i,round(gsector_returns[gsector_returns['GSECTOR'] == i].RetLate2020.mean(), 3)))

In [ ]:
late_2020_gsector = pd.DataFrame(list8, columns=['GSECTOR', 'AVG'])
late_2020_gsector[late_2020_gsector['AVG'] == max(late_2020_gsector['AVG'])]

,GSECTOR,AVG
5,25,1.541


The highest return industry for late 2020 is Consumer Discretionary. Since all the industries have the positive average return for late 2020, it seems like the economy was gradually recovering from the covid hit. People started to going out, traveling and shopping. Because of this, the demand of consumer goods went up a lot. That's why the Consumer Discretionary industry were doing great for the late 2020.





In [ ]:
late_2020_gsector[late_2020_gsector['AVG'] == min(late_2020_gsector['AVG'])]

,GSECTOR,AVG
9,55,0.181


Same as the GGROUP industry code, the lowest return industry of the GSECTOR code for late 2020 is also Utilities. This makes economic sense because utilities is sensitive to the health of the economy. Covid hits the economy very bad, lots of factries shut down or experienced downturn due to the prolonged lockdown. Malls were forced to close to avoid the spread of covid. In addition, airports and railways are major consumers of this industry, and the demand from them dropped sharply due to the lockdowns, which led to the over revenue decrease in the industry.

### Summary

**Early 2020**

NAICS 
*   Lowest return : Oil and Gas Extraction

*   Highest return : In-Vitro Diagnostic Substance Manufacturingc

SIC
*   Lowest return : Oil and Gas Extraction

*   Highest return : In Vitro and In Vivo Diagnostic Substances

GGROUP
*   Lowest return : Energy

*   Highest return : Telecommunication Services

GSECTOR
*   Lowest return : Energy

*   Highest return : Heath Care


---
**Late 2020**

NAICS 
*   Lowest return : Deep Sea Freight Transportation

*   Highest return : Oil and Gas Extraction 

SIC
*   Lowest return : Electric and Other Services Combined

*   Highest return : Retail-Auto Dealers & Gasoline Stations

GGROUP
*   Lowest return : Utilities

*   Highest return : Automobiles & Components

GSECTOR
*   Lowest return : Utilities

*   Highest return : Consumer Discretionary


## Regression Analysis

### RetEarly2020 (compare with the market ratios)

In [ ]:
mr = pd.read_csv('temp2.csv')

#### NAICS

In [ ]:
dd1 = pd.get_dummies(naics_25_returns, columns=['NAICS'])

In [ ]:
mr.rename(columns={'tic': 'TICKER'}, inplace=True)
new1 = pd.merge(dd1, mr, on='TICKER')

In [ ]:
new1['book/price'] = new1['ceq']/(new1['prcc_c']*new1['csho'])
new1['e/p'] = new1['ni']/(new1['prcc_c']*new1['csho'])
new1['ebit/p'] = new1['ebit']/(new1['prcc_c']*new1['csho'])
new1['sales/p'] = new1['sale']/(new1['prcc_c']*new1['csho'])

In [ ]:
new1.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new1['book/price'] = winsorize(new1['book/price'], (0.01, 0.01))
new1['e/p'] = winsorize(new1['e/p'], (0.01, 0.01))
new1['ebit/p'] = winsorize(new1['ebit/p'], (0.01, 0.01))
new1['sales/p'] = winsorize(new1['sales/p'], (0.01, 0.01))

In [ ]:
new1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1313
Data columns (total 32 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        1314 non-null   object 
 1   RetEarly2020  1314 non-null   float64
 2   RetLate2020   1314 non-null   float64
 3   NAICS_2111    1314 non-null   uint8  
 4   NAICS_2211    1314 non-null   uint8  
 5   NAICS_22111   1314 non-null   uint8  
 6   NAICS_221210  1314 non-null   uint8  
 7   NAICS_325412  1314 non-null   uint8  
 8   NAICS_325413  1314 non-null   uint8  
 9   NAICS_325414  1314 non-null   uint8  
 10  NAICS_334413  1314 non-null   uint8  
 11  NAICS_334510  1314 non-null   uint8  
 12  NAICS_334516  1314 non-null   uint8  
 13  NAICS_339112  1314 non-null   uint8  
 14  NAICS_339113  1314 non-null   uint8  
 15  NAICS_483111  1314 non-null   uint8  
 16  NAICS_511210  1314 non-null   uint8  
 17  NAICS_518210  1314 non-null   uint8  
 18  NAICS_519130  1314 non-null 

In [ ]:
subset = new1.iloc[:, 28: 32]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1313
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  1314 non-null   float64
 1   e/p         1314 non-null   float64
 2   ebit/p      1314 non-null   float64
 3   sales/p     1314 non-null   float64
dtypes: float64(4)
memory usage: 51.3 KB


In [ ]:
subset1 = new1.iloc[:, 3:28]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1313
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   NAICS_2111    1314 non-null   uint8
 1   NAICS_2211    1314 non-null   uint8
 2   NAICS_22111   1314 non-null   uint8
 3   NAICS_221210  1314 non-null   uint8
 4   NAICS_325412  1314 non-null   uint8
 5   NAICS_325413  1314 non-null   uint8
 6   NAICS_325414  1314 non-null   uint8
 7   NAICS_334413  1314 non-null   uint8
 8   NAICS_334510  1314 non-null   uint8
 9   NAICS_334516  1314 non-null   uint8
 10  NAICS_339112  1314 non-null   uint8
 11  NAICS_339113  1314 non-null   uint8
 12  NAICS_483111  1314 non-null   uint8
 13  NAICS_511210  1314 non-null   uint8
 14  NAICS_518210  1314 non-null   uint8
 15  NAICS_519130  1314 non-null   uint8
 16  NAICS_522110  1314 non-null   uint8
 17  NAICS_522120  1314 non-null   uint8
 18  NAICS_523920  1314 non-null   uint8
 19  NAICS_524126  1314 non-null

In [ ]:
Y = new1['RetEarly2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     52.06
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           9.89e-41
Time:                        19:26:06   Log-Likelihood:                -482.90
No. Observations:                1314   AIC:                             975.8
Df Residuals:                    1309   BIC:                             1002.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1718      0.016    -10.600      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     7.769
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           5.04e-25
Time:                        19:26:06   Log-Likelihood:                -491.13
No. Observations:                1314   AIC:                             1032.
Df Residuals:                    1289   BIC:                             1162.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
NAICS_2111      -0.6299      0.077     -8.131   

In [ ]:
X2 = new1.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     11.10
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.40e-43
Time:                        19:26:07   Log-Likelihood:                -437.63
No. Observations:                1314   AIC:                             933.3
Df Residuals:                    1285   BIC:                             1083.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.1667      0.021     -7.867   

#### SIC

In [ ]:
dd2 = pd.get_dummies(sic_25_returns, columns=['SIC'])

In [ ]:
new2 = pd.merge(dd2, mr, on='TICKER')

In [ ]:
new2['book/price'] = new2['ceq']/(new2['prcc_c']*new2['csho'])
new2['e/p'] = new2['ni']/(new2['prcc_c']*new2['csho'])
new2['ebit/p'] = new2['ebit']/(new2['prcc_c']*new2['csho'])
new2['sales/p'] = new2['sale']/(new2['prcc_c']*new2['csho'])

In [ ]:
new2.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new2['book/price'] = winsorize(new2['book/price'], (0.01, 0.01))
new2['e/p'] = winsorize(new2['e/p'], (0.01, 0.01))
new2['ebit/p'] = winsorize(new2['ebit/p'], (0.01, 0.01))
new2['sales/p'] = winsorize(new2['sales/p'], (0.01, 0.01))

In [ ]:
subset = new2.iloc[:, 28: 32]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393 entries, 0 to 1392
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  1393 non-null   float64
 1   e/p         1393 non-null   float64
 2   ebit/p      1393 non-null   float64
 3   sales/p     1393 non-null   float64
dtypes: float64(4)
memory usage: 54.4 KB


In [ ]:
subset1 = new2.iloc[:, 3:28]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393 entries, 0 to 1392
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   SIC_1311  1393 non-null   uint8
 1   SIC_1531  1393 non-null   uint8
 2   SIC_2834  1393 non-null   uint8
 3   SIC_2835  1393 non-null   uint8
 4   SIC_2836  1393 non-null   uint8
 5   SIC_3674  1393 non-null   uint8
 6   SIC_3714  1393 non-null   uint8
 7   SIC_3826  1393 non-null   uint8
 8   SIC_3841  1393 non-null   uint8
 9   SIC_3842  1393 non-null   uint8
 10  SIC_3845  1393 non-null   uint8
 11  SIC_4911  1393 non-null   uint8
 12  SIC_4931  1393 non-null   uint8
 13  SIC_5500  1393 non-null   uint8
 14  SIC_5812  1393 non-null   uint8
 15  SIC_6020  1393 non-null   uint8
 16  SIC_6035  1393 non-null   uint8
 17  SIC_6141  1393 non-null   uint8
 18  SIC_6282  1393 non-null   uint8
 19  SIC_6331  1393 non-null   uint8
 20  SIC_6798  1393 non-null   uint8
 21  SIC_7370  1393 non-null   uint8
 22  

In [ ]:
Y = new2['RetEarly2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     59.19
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           3.45e-46
Time:                        19:26:07   Log-Likelihood:                -491.73
No. Observations:                1393   AIC:                             993.5
Df Residuals:                    1388   BIC:                             1020.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1704      0.015    -11.319      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     9.249
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           3.94e-31
Time:                        19:26:07   Log-Likelihood:                -496.70
No. Observations:                1393   AIC:                             1043.
Df Residuals:                    1368   BIC:                             1174.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SIC_1311      -0.6734      0.058    -11.584      0.0

In [ ]:
X2 = new2.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.214
Model:                            OLS   Adj. R-squared:                  0.198
Method:                 Least Squares   F-statistic:                     13.25
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.44e-53
Time:                        19:26:07   Log-Likelihood:                -433.85
No. Observations:                1393   AIC:                             925.7
Df Residuals:                    1364   BIC:                             1078.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1666      0.019     -8.663      0.0

#### GGROUP

In [ ]:
dd3 = pd.get_dummies(ggroup_returns, columns=['GGROUP'])

In [ ]:
new3 = pd.merge(dd3, mr, on='TICKER')

In [ ]:
new3['book/price'] = new3['ceq']/(new3['prcc_c']*new3['csho'])
new3['e/p'] = new3['ni']/(new3['prcc_c']*new3['csho'])
new3['ebit/p'] = new3['ebit']/(new3['prcc_c']*new3['csho'])
new3['sales/p'] = new3['sale']/(new3['prcc_c']*new3['csho'])

In [ ]:
new3.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new3['book/price'] = winsorize(new3['book/price'], (0.01, 0.01))
new3['e/p'] = winsorize(new3['e/p'], (0.01, 0.01))
new3['ebit/p'] = winsorize(new3['ebit/p'], (0.01, 0.01))
new3['sales/p'] = winsorize(new3['sales/p'], (0.01, 0.01))

In [ ]:
subset = new3.iloc[:, 27: 31]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  2666 non-null   float64
 1   e/p         2666 non-null   float64
 2   ebit/p      2666 non-null   float64
 3   sales/p     2666 non-null   float64
dtypes: float64(4)
memory usage: 104.1 KB


In [ ]:
subset1 = new3.iloc[:, 3:27]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   GGROUP_1010  2666 non-null   uint8
 1   GGROUP_1510  2666 non-null   uint8
 2   GGROUP_2010  2666 non-null   uint8
 3   GGROUP_2020  2666 non-null   uint8
 4   GGROUP_2030  2666 non-null   uint8
 5   GGROUP_2510  2666 non-null   uint8
 6   GGROUP_2520  2666 non-null   uint8
 7   GGROUP_2530  2666 non-null   uint8
 8   GGROUP_2550  2666 non-null   uint8
 9   GGROUP_3010  2666 non-null   uint8
 10  GGROUP_3020  2666 non-null   uint8
 11  GGROUP_3030  2666 non-null   uint8
 12  GGROUP_3510  2666 non-null   uint8
 13  GGROUP_3520  2666 non-null   uint8
 14  GGROUP_4010  2666 non-null   uint8
 15  GGROUP_4020  2666 non-null   uint8
 16  GGROUP_4030  2666 non-null   uint8
 17  GGROUP_4510  2666 non-null   uint8
 18  GGROUP_4520  2666 non-null   uint8
 19  GGROUP_4530  2666 non-null   uint8
 20  GGROUP_5

In [ ]:
Y = new3['RetEarly2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     74.80
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           3.66e-60
Time:                        19:26:07   Log-Likelihood:                -545.33
No. Observations:                2666   AIC:                             1101.
Df Residuals:                    2661   BIC:                             1130.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2067      0.009    -23.164      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                     17.25
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.59e-64
Time:                        19:26:07   Log-Likelihood:                -500.90
No. Observations:                2666   AIC:                             1050.
Df Residuals:                    2642   BIC:                             1191.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
GGROUP_1010    -0.5807      0.029    -20.192      

In [ ]:
X2 = new3.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     20.50
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.62e-89
Time:                        19:26:07   Log-Likelihood:                -433.49
No. Observations:                2666   AIC:                             923.0
Df Residuals:                    2638   BIC:                             1088.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.2129      0.011    -20.004      

#### GSECTOR

In [ ]:
dd4 = pd.get_dummies(gsector_returns, columns=['GSECTOR'])

In [ ]:
new4 = pd.merge(dd4, mr, on='TICKER')

In [ ]:
new4['book/price'] = new4['ceq']/(new4['prcc_c']*new4['csho'])
new4['e/p'] = new4['ni']/(new4['prcc_c']*new4['csho'])
new4['ebit/p'] = new4['ebit']/(new4['prcc_c']*new4['csho'])
new4['sales/p'] = new4['sale']/(new4['prcc_c']*new4['csho'])

In [ ]:
new4.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new4['book/price'] = winsorize(new4['book/price'], (0.01, 0.01))
new4['e/p'] = winsorize(new4['e/p'], (0.01, 0.01))
new4['ebit/p'] = winsorize(new4['ebit/p'], (0.01, 0.01))
new4['sales/p'] = winsorize(new4['sales/p'], (0.01, 0.01))

In [ ]:
new4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        2666 non-null   object 
 1   RetEarly2020  2666 non-null   float64
 2   RetLate2020   2666 non-null   float64
 3   GSECTOR_10    2666 non-null   uint8  
 4   GSECTOR_15    2666 non-null   uint8  
 5   GSECTOR_20    2666 non-null   uint8  
 6   GSECTOR_25    2666 non-null   uint8  
 7   GSECTOR_30    2666 non-null   uint8  
 8   GSECTOR_35    2666 non-null   uint8  
 9   GSECTOR_40    2666 non-null   uint8  
 10  GSECTOR_45    2666 non-null   uint8  
 11  GSECTOR_50    2666 non-null   uint8  
 12  GSECTOR_55    2666 non-null   uint8  
 13  GSECTOR_60    2666 non-null   uint8  
 14  book/price    2666 non-null   float64
 15  e/p           2666 non-null   float64
 16  ebit/p        2666 non-null   float64
 17  sales/p       2666 non-null   float64
dtypes: float64(6), object(1), ui

In [ ]:
subset = new4.iloc[:, 14: 18]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  2666 non-null   float64
 1   e/p         2666 non-null   float64
 2   ebit/p      2666 non-null   float64
 3   sales/p     2666 non-null   float64
dtypes: float64(4)
memory usage: 104.1 KB


In [ ]:
subset1 = new4.iloc[:, 3:14]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   GSECTOR_10  2666 non-null   uint8
 1   GSECTOR_15  2666 non-null   uint8
 2   GSECTOR_20  2666 non-null   uint8
 3   GSECTOR_25  2666 non-null   uint8
 4   GSECTOR_30  2666 non-null   uint8
 5   GSECTOR_35  2666 non-null   uint8
 6   GSECTOR_40  2666 non-null   uint8
 7   GSECTOR_45  2666 non-null   uint8
 8   GSECTOR_50  2666 non-null   uint8
 9   GSECTOR_55  2666 non-null   uint8
 10  GSECTOR_60  2666 non-null   uint8
dtypes: uint8(11)
memory usage: 49.5 KB


In [ ]:
Y = new4['RetEarly2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     74.80
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           3.66e-60
Time:                        19:26:08   Log-Likelihood:                -545.33
No. Observations:                2666   AIC:                             1101.
Df Residuals:                    2661   BIC:                             1130.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2067      0.009    -23.164      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     35.82
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           2.95e-66
Time:                        19:26:08   Log-Likelihood:                -518.68
No. Observations:                2666   AIC:                             1059.
Df Residuals:                    2655   BIC:                             1124.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
GSECTOR_10    -0.5807      0.029    -20.107      0.0

In [ ]:
X2 = new4.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:           RetEarly2020   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     36.39
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           8.86e-91
Time:                        19:26:08   Log-Likelihood:                -453.04
No. Observations:                2666   AIC:                             936.1
Df Residuals:                    2651   BIC:                             1024.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2180      0.010    -22.212      0.0

### RetLate2020 (compare with the market ratios)

#### NAICS

In [ ]:
dd5 = pd.get_dummies(naics_25_returns, columns=['NAICS'])

In [ ]:
new5 = pd.merge(dd5, mr, on='TICKER')

In [ ]:
new5['book/price'] = new5['ceq']/(new5['prcc_c']*new5['csho'])
new5['e/p'] = new5['ni']/(new5['prcc_c']*new5['csho'])
new5['ebit/p'] = new5['ebit']/(new5['prcc_c']*new5['csho'])
new5['sales/p'] = new5['sale']/(new5['prcc_c']*new5['csho'])

In [ ]:
new5.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new5['book/price'] = winsorize(new5['book/price'], (0.01, 0.01))
new5['e/p'] = winsorize(new5['e/p'], (0.01, 0.01))
new5['ebit/p'] = winsorize(new5['ebit/p'], (0.01, 0.01))
new5['sales/p'] = winsorize(new5['sales/p'], (0.01, 0.01))

In [ ]:
subset = new5.iloc[:, 28: 32]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1313
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  1314 non-null   float64
 1   e/p         1314 non-null   float64
 2   ebit/p      1314 non-null   float64
 3   sales/p     1314 non-null   float64
dtypes: float64(4)
memory usage: 51.3 KB


In [ ]:
subset1 = new5.iloc[:, 3:28]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1313
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   NAICS_2111    1314 non-null   uint8
 1   NAICS_2211    1314 non-null   uint8
 2   NAICS_22111   1314 non-null   uint8
 3   NAICS_221210  1314 non-null   uint8
 4   NAICS_325412  1314 non-null   uint8
 5   NAICS_325413  1314 non-null   uint8
 6   NAICS_325414  1314 non-null   uint8
 7   NAICS_334413  1314 non-null   uint8
 8   NAICS_334510  1314 non-null   uint8
 9   NAICS_334516  1314 non-null   uint8
 10  NAICS_339112  1314 non-null   uint8
 11  NAICS_339113  1314 non-null   uint8
 12  NAICS_483111  1314 non-null   uint8
 13  NAICS_511210  1314 non-null   uint8
 14  NAICS_518210  1314 non-null   uint8
 15  NAICS_519130  1314 non-null   uint8
 16  NAICS_522110  1314 non-null   uint8
 17  NAICS_522120  1314 non-null   uint8
 18  NAICS_523920  1314 non-null   uint8
 19  NAICS_524126  1314 non-null

In [ ]:
Y = new5['RetLate2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     20.03
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           5.07e-16
Time:                        19:26:08   Log-Likelihood:                -1809.0
No. Observations:                1314   AIC:                             3628.
Df Residuals:                    1309   BIC:                             3654.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6793      0.044     15.280      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.127
Method:                 Least Squares   F-statistic:                     8.958
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           8.34e-30
Time:                        19:26:08   Log-Likelihood:                -1746.7
No. Observations:                1314   AIC:                             3543.
Df Residuals:                    1289   BIC:                             3673.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
NAICS_2111       1.7285      0.201      8.581   

In [ ]:
X2 = new5.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     9.432
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           4.69e-36
Time:                        19:26:08   Log-Likelihood:                -1725.2
No. Observations:                1314   AIC:                             3508.
Df Residuals:                    1285   BIC:                             3659.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.6158      0.056     10.906   

#### SIC

In [ ]:
dd6 = pd.get_dummies(sic_25_returns, columns=['SIC'])

In [ ]:
new6 = pd.merge(dd6, mr, on='TICKER')

In [ ]:
new6['book/price'] = new6['ceq']/(new6['prcc_c']*new6['csho'])
new6['e/p'] = new6['ni']/(new6['prcc_c']*new6['csho'])
new6['ebit/p'] = new6['ebit']/(new6['prcc_c']*new6['csho'])
new6['sales/p'] = new6['sale']/(new6['prcc_c']*new6['csho'])

In [ ]:
new6.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new6['book/price'] = winsorize(new6['book/price'], (0.01, 0.01))
new6['e/p'] = winsorize(new6['e/p'], (0.01, 0.01))
new6['ebit/p'] = winsorize(new6['ebit/p'], (0.01, 0.01))
new6['sales/p'] = winsorize(new6['sales/p'], (0.01, 0.01))

In [ ]:
subset = new6.iloc[:, 28: 32]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393 entries, 0 to 1392
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  1393 non-null   float64
 1   e/p         1393 non-null   float64
 2   ebit/p      1393 non-null   float64
 3   sales/p     1393 non-null   float64
dtypes: float64(4)
memory usage: 54.4 KB


In [ ]:
subset1 = new6.iloc[:, 3:28]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393 entries, 0 to 1392
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   SIC_1311  1393 non-null   uint8
 1   SIC_1531  1393 non-null   uint8
 2   SIC_2834  1393 non-null   uint8
 3   SIC_2835  1393 non-null   uint8
 4   SIC_2836  1393 non-null   uint8
 5   SIC_3674  1393 non-null   uint8
 6   SIC_3714  1393 non-null   uint8
 7   SIC_3826  1393 non-null   uint8
 8   SIC_3841  1393 non-null   uint8
 9   SIC_3842  1393 non-null   uint8
 10  SIC_3845  1393 non-null   uint8
 11  SIC_4911  1393 non-null   uint8
 12  SIC_4931  1393 non-null   uint8
 13  SIC_5500  1393 non-null   uint8
 14  SIC_5812  1393 non-null   uint8
 15  SIC_6020  1393 non-null   uint8
 16  SIC_6035  1393 non-null   uint8
 17  SIC_6141  1393 non-null   uint8
 18  SIC_6282  1393 non-null   uint8
 19  SIC_6331  1393 non-null   uint8
 20  SIC_6798  1393 non-null   uint8
 21  SIC_7370  1393 non-null   uint8
 22  

In [ ]:
Y = new6['RetLate2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     29.01
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           3.46e-23
Time:                        19:26:09   Log-Likelihood:                -1901.8
No. Observations:                1393   AIC:                             3814.
Df Residuals:                    1388   BIC:                             3840.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6820      0.041     16.467      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     9.746
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           3.88e-33
Time:                        19:26:09   Log-Likelihood:                -1847.8
No. Observations:                1393   AIC:                             3746.
Df Residuals:                    1368   BIC:                             3877.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SIC_1311       1.3432      0.153      8.760      0.0

In [ ]:
X2 = new6.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     10.56
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.98e-41
Time:                        19:26:09   Log-Likelihood:                -1821.1
No. Observations:                1393   AIC:                             3700.
Df Residuals:                    1364   BIC:                             3852.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6620      0.052     12.713      0.0

#### GGROUP

In [ ]:
dd7 = pd.get_dummies(ggroup_returns, columns=['GGROUP'])

In [ ]:
new7 = pd.merge(dd7, mr, on='TICKER')

In [ ]:
new7['book/price'] = new7['ceq']/(new7['prcc_c']*new7['csho'])
new7['e/p'] = new7['ni']/(new7['prcc_c']*new7['csho'])
new7['ebit/p'] = new7['ebit']/(new7['prcc_c']*new7['csho'])
new7['sales/p'] = new7['sale']/(new7['prcc_c']*new7['csho'])

In [ ]:
new7.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new7['book/price'] = winsorize(new7['book/price'], (0.01, 0.01))
new7['e/p'] = winsorize(new7['e/p'], (0.01, 0.01))
new7['ebit/p'] = winsorize(new7['ebit/p'], (0.01, 0.01))
new7['sales/p'] = winsorize(new7['sales/p'], (0.01, 0.01))

In [ ]:
subset = new7.iloc[:, 27: 31]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  2666 non-null   float64
 1   e/p         2666 non-null   float64
 2   ebit/p      2666 non-null   float64
 3   sales/p     2666 non-null   float64
dtypes: float64(4)
memory usage: 104.1 KB


In [ ]:
subset1 = new7.iloc[:, 3:27]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   GGROUP_1010  2666 non-null   uint8
 1   GGROUP_1510  2666 non-null   uint8
 2   GGROUP_2010  2666 non-null   uint8
 3   GGROUP_2020  2666 non-null   uint8
 4   GGROUP_2030  2666 non-null   uint8
 5   GGROUP_2510  2666 non-null   uint8
 6   GGROUP_2520  2666 non-null   uint8
 7   GGROUP_2530  2666 non-null   uint8
 8   GGROUP_2550  2666 non-null   uint8
 9   GGROUP_3010  2666 non-null   uint8
 10  GGROUP_3020  2666 non-null   uint8
 11  GGROUP_3030  2666 non-null   uint8
 12  GGROUP_3510  2666 non-null   uint8
 13  GGROUP_3520  2666 non-null   uint8
 14  GGROUP_4010  2666 non-null   uint8
 15  GGROUP_4020  2666 non-null   uint8
 16  GGROUP_4030  2666 non-null   uint8
 17  GGROUP_4510  2666 non-null   uint8
 18  GGROUP_4520  2666 non-null   uint8
 19  GGROUP_4530  2666 non-null   uint8
 20  GGROUP_5

In [ ]:
Y = new7['RetLate2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     49.98
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.30e-40
Time:                        19:26:09   Log-Likelihood:                -3876.9
No. Observations:                2666   AIC:                             7764.
Df Residuals:                    2661   BIC:                             7793.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7715      0.031     24.775      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.110
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     14.14
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.16e-51
Time:                        19:26:09   Log-Likelihood:                -3818.7
No. Observations:                2666   AIC:                             7685.
Df Residuals:                    2642   BIC:                             7827.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
GGROUP_1010     1.0175      0.100     10.192      

In [ ]:
X2 = new7.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     17.22
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           3.43e-74
Time:                        19:26:09   Log-Likelihood:                -3757.1
No. Observations:                2666   AIC:                             7570.
Df Residuals:                    2638   BIC:                             7735.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.7571      0.037     20.451      

#### GSECTOR

In [ ]:
dd8 = pd.get_dummies(gsector_returns, columns=['GSECTOR'])

In [ ]:
new8 = pd.merge(dd8, mr, on='TICKER')

In [ ]:
new8['book/price'] = new8['ceq']/(new8['prcc_c']*new8['csho'])
new8['e/p'] = new8['ni']/(new8['prcc_c']*new8['csho'])
new8['ebit/p'] = new8['ebit']/(new8['prcc_c']*new8['csho'])
new8['sales/p'] = new8['sale']/(new8['prcc_c']*new8['csho'])

In [ ]:
new8.drop(['gvkey', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'ceq', 'csho', 
            'ebit', 'ni', 'sale', 'costat', 'prcc_c'], axis=1, inplace=True)

In [ ]:
#winsorize the ratio variables at 1% and 99% values
new8['book/price'] = winsorize(new8['book/price'], (0.01, 0.01))
new8['e/p'] = winsorize(new8['e/p'], (0.01, 0.01))
new8['ebit/p'] = winsorize(new8['ebit/p'], (0.01, 0.01))
new8['sales/p'] = winsorize(new8['sales/p'], (0.01, 0.01))

In [ ]:
new8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TICKER        2666 non-null   object 
 1   RetEarly2020  2666 non-null   float64
 2   RetLate2020   2666 non-null   float64
 3   GSECTOR_10    2666 non-null   uint8  
 4   GSECTOR_15    2666 non-null   uint8  
 5   GSECTOR_20    2666 non-null   uint8  
 6   GSECTOR_25    2666 non-null   uint8  
 7   GSECTOR_30    2666 non-null   uint8  
 8   GSECTOR_35    2666 non-null   uint8  
 9   GSECTOR_40    2666 non-null   uint8  
 10  GSECTOR_45    2666 non-null   uint8  
 11  GSECTOR_50    2666 non-null   uint8  
 12  GSECTOR_55    2666 non-null   uint8  
 13  GSECTOR_60    2666 non-null   uint8  
 14  book/price    2666 non-null   float64
 15  e/p           2666 non-null   float64
 16  ebit/p        2666 non-null   float64
 17  sales/p       2666 non-null   float64
dtypes: float64(6), object(1), ui

In [ ]:
subset = new8.iloc[:, 14: 18]
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   book/price  2666 non-null   float64
 1   e/p         2666 non-null   float64
 2   ebit/p      2666 non-null   float64
 3   sales/p     2666 non-null   float64
dtypes: float64(4)
memory usage: 104.1 KB


In [ ]:
subset1 = new8.iloc[:, 3:14]
subset1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666 entries, 0 to 2665
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   GSECTOR_10  2666 non-null   uint8
 1   GSECTOR_15  2666 non-null   uint8
 2   GSECTOR_20  2666 non-null   uint8
 3   GSECTOR_25  2666 non-null   uint8
 4   GSECTOR_30  2666 non-null   uint8
 5   GSECTOR_35  2666 non-null   uint8
 6   GSECTOR_40  2666 non-null   uint8
 7   GSECTOR_45  2666 non-null   uint8
 8   GSECTOR_50  2666 non-null   uint8
 9   GSECTOR_55  2666 non-null   uint8
 10  GSECTOR_60  2666 non-null   uint8
dtypes: uint8(11)
memory usage: 49.5 KB


In [ ]:
Y = new8['RetLate2020']
X = subset
X = sm.add_constant(X)
model_market_early = sm.OLS(Y, X).fit()
print(model_market_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     49.98
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.30e-40
Time:                        19:26:09   Log-Likelihood:                -3876.9
No. Observations:                2666   AIC:                             7764.
Df Residuals:                    2661   BIC:                             7793.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7715      0.031     24.775      0.0

In [ ]:
X1 = subset1
model_naics_early = sm.OLS(Y, X1).fit()
print(model_naics_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     27.52
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           1.43e-50
Time:                        19:26:09   Log-Likelihood:                -3841.9
No. Observations:                2666   AIC:                             7706.
Df Residuals:                    2655   BIC:                             7771.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
GSECTOR_10     1.0175      0.100     10.128      0.0

In [ ]:
X2 = new8.drop(columns=['TICKER', 'RetEarly2020','RetLate2020'])
X2 = sm.add_constant(X2)
m_early = sm.OLS(Y, X2).fit()
print(m_early.summary())

                            OLS Regression Results                            
Dep. Variable:            RetLate2020   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     29.24
Date:                Wed, 07 Jul 2021   Prob (F-statistic):           9.59e-73
Time:                        19:26:09   Log-Likelihood:                -3782.0
No. Observations:                2666   AIC:                             7594.
Df Residuals:                    2651   BIC:                             7682.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6780      0.034     19.820      0.0

# Conclusion

The SIC industry identifier has the highest explanatory power among the four different industry identifiers for both ealy 2020 returns and late 2020 returns. 

In general, industry variables have more explanatory power than the market ratio variables. When the industry fixed effects are combined with the market ratios, the explanatory power gets boosted and can better explain the differences in Stock Returns for early and late 2020.